In [1]:
import platform
import os

system = platform.system().lower().strip()
if system in ['darwin', 'windows']:
    path_root = '.'
elif system == 'linux':
    from google.colab import drive
    drive.mount('/content/drive')

    path_root = './drive/MyDrive/code/03.crawler/39.OEMPartsOnline'
else:
    print('fuck')

path_root

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


'./drive/MyDrive/code/03.crawler/39.OEMPartsOnline'

In [2]:
import pandas as pd

# = = =  = = =  = = =

import requests

import sys
sys.path.append(os.path.join(path_root, '../00.Tools'))
from crawler_configuration_oversea import get_header, get_proxy

from bs4 import BeautifulSoup
from lxml import etree

from urllib.parse import urlparse

In [3]:
print('Data Loading...')
list_series_error, df_input = [], pd.read_excel(os.path.join(path_root, './file/1.page.xlsx'),
                                                header=0,
                                                dtype=str).fillna('')
print('Loading Done !')
print()

df_input['No'] = df_input['No'].astype(int)
df_input['SKU Count'] = df_input['SKU Count'].astype(int)
df_input['Page'] = df_input['Page'].astype(int)

crawler_remain = len(df_input)

print(f'总数量：{crawler_remain}')
print()

df_input

Data Loading...
Loading Done !

总数量：1



,No,Url,SKU Count,Page
0,1,https://toyota.oempartsonline.com/search?searc...,12123,3


In [4]:
crawler_index, crawler_series = next(df_input.iterrows())

crawler_index, crawler_series

(0,
 No                                                           1
 Url          https://toyota.oempartsonline.com/search?searc...
 SKU Count                                                12123
 Page                                                         3
 Name: 0, dtype: object)

In [5]:
import time

list_url = []
for crawler_cycle in range(5):

    for page in range(1, crawler_series['Page'] + 1):

        crawler_retry = 0
        while True:
            crawler_retry += 1
            print(crawler_cycle, page, crawler_retry)
            try:
                url_request = f'''{crawler_series['Url']}&page={page}'''

                resp = requests.get(url_request,
                                    headers=get_header(),
                                    proxies=get_proxy(),
                                    timeout=(5, 15))

                if resp.status_code == 200:
                    soup = BeautifulSoup(resp.text, 'lxml')
                    html = etree.HTML(str(soup))

                    list_href = html.xpath('//div[@class="catalog-product marketplace-product row"]/div[@class="product-details-col col-md-10 row"]/div[@class="col-xs-12 col-md-8"]/div[@class="product-partnum-category"]/div[@class="catalog-product-id"]/a/@href') + html.xpath('//div[@class="catalog-product row"]/div[@class="product-details-col col-xs-7"]/div[@class="product-partnum-category"]/div[@class="catalog-product-id"]/a/@href')
                    if list_href:
                        if crawler_cycle == 0 and page == 1:
                            with open(os.path.join(path_root, './url.html'), 'w', encoding='UTF-8') as file:
                                file.write(str(soup))
                        break
            except requests.exceptions.ProxyError as e:
                if '456 CN Client Forbidden' in str(e):
                    print('Long live the People\'s Republic of China!')
                    break
            except KeyboardInterrupt:
                break
            except:
                time.sleep(3)
                continue

        # = = =  = = =  = = =

        url_parse = urlparse(crawler_series['Url'])
        list_href = [f'''{url_parse.scheme}://{url_parse.netloc}{href.split('?')[0].strip()}''' for href in list_href]

        # = = =  = = =  = = =

        list_url.extend(list_href)
        list_url = list(dict.fromkeys(list_url))

        # = = =  = = =  = = =

        print(f''' - {len(list_url)}/{crawler_series['SKU Count']}''')

        # = = =  = = =  = = =

        if len(list_url) == crawler_series['SKU Count']:
            break

    if len(list_url) == crawler_series['SKU Count']:
        break

len(list_url), list_url

0 1 1
 - 18/12123
0 2 1
 - 36/12123
0 3 1
 - 54/12123
1 1 1
 - 54/12123
1 2 1
 - 54/12123
1 3 1
 - 54/12123
2 1 1
 - 54/12123
2 2 1
 - 54/12123
2 3 1
 - 54/12123
3 1 1
 - 54/12123
3 2 1
3 2 2
 - 54/12123
3 3 1
 - 54/12123
4 1 1
 - 54/12123
4 2 1
 - 54/12123
4 3 1
 - 54/12123


(54,
 ['https://toyota.oempartsonline.com/oem-parts/toyota-fuel-injector-232092507001',
  'https://toyota.oempartsonline.com/oem-parts/toyota-fuel-injector-232092507002',
  'https://toyota.oempartsonline.com/oem-parts/toyota-fuel-injector-2320939335',
  'https://toyota.oempartsonline.com/oem-parts/toyota-fuel-injector-2320946090',
  'https://toyota.oempartsonline.com/oem-parts/toyota-fuel-injector-2320903010',
  'https://toyota.oempartsonline.com/oem-parts/toyota-fuel-injector-2320909100',
  'https://toyota.oempartsonline.com/oem-parts/toyota-fuel-injector-2320909140',
  'https://toyota.oempartsonline.com/oem-parts/toyota-fuel-injector-2320909151',
  'https://toyota.oempartsonline.com/oem-parts/toyota-fuel-injector-2320909175',
  'https://toyota.oempartsonline.com/oem-parts/toyota-fuel-injector-2320909330',
  'https://toyota.oempartsonline.com/oem-parts/toyota-fuel-injector-232090f010',
  'https://toyota.oempartsonline.com/oem-parts/toyota-fuel-injector-232090f020',
  'https://toyota.o

In [6]:
list_url

['https://toyota.oempartsonline.com/oem-parts/toyota-fuel-injector-232092507001',
 'https://toyota.oempartsonline.com/oem-parts/toyota-fuel-injector-232092507002',
 'https://toyota.oempartsonline.com/oem-parts/toyota-fuel-injector-2320939335',
 'https://toyota.oempartsonline.com/oem-parts/toyota-fuel-injector-2320946090',
 'https://toyota.oempartsonline.com/oem-parts/toyota-fuel-injector-2320903010',
 'https://toyota.oempartsonline.com/oem-parts/toyota-fuel-injector-2320909100',
 'https://toyota.oempartsonline.com/oem-parts/toyota-fuel-injector-2320909140',
 'https://toyota.oempartsonline.com/oem-parts/toyota-fuel-injector-2320909151',
 'https://toyota.oempartsonline.com/oem-parts/toyota-fuel-injector-2320909175',
 'https://toyota.oempartsonline.com/oem-parts/toyota-fuel-injector-2320909330',
 'https://toyota.oempartsonline.com/oem-parts/toyota-fuel-injector-232090f010',
 'https://toyota.oempartsonline.com/oem-parts/toyota-fuel-injector-232090f020',
 'https://toyota.oempartsonline.com/

In [7]:
print('Data Outputting...')
df_correct = pd.DataFrame({'No': range(1, len(list_url) + 1),
                           'Url': list_url})
df_correct.to_excel(os.path.join(path_root, './test_url.xlsx'), index=False)

df_correct

Data Outputting...


,No,Url
0,1,https://toyota.oempartsonline.com/oem-parts/to...
1,2,https://toyota.oempartsonline.com/oem-parts/to...
2,3,https://toyota.oempartsonline.com/oem-parts/to...
3,4,https://toyota.oempartsonline.com/oem-parts/to...
4,5,https://toyota.oempartsonline.com/oem-parts/to...
5,6,https://toyota.oempartsonline.com/oem-parts/to...
6,7,https://toyota.oempartsonline.com/oem-parts/to...
7,8,https://toyota.oempartsonline.com/oem-parts/to...
8,9,https://toyota.oempartsonline.com/oem-parts/to...
9,10,https://toyota.oempartsonline.com/oem-parts/to...


In [8]:
print('Output Finished !')
print()

Output Finished !



In [9]:
crawler_series['Request_Url'] = url_request

list_series_error.append(crawler_series)

list_series_error

[No                                                             1
 Url            https://toyota.oempartsonline.com/search?searc...
 SKU Count                                                  12123
 Page                                                           3
 Request_Url    https://toyota.oempartsonline.com/search?searc...
 Name: 0, dtype: object]

In [10]:
if list_series_error:
    df_error = pd.DataFrame(list_series_error).sort_values(by=['No'],
                                                           ascending=[True],
                                                           ignore_index=True)
    df_error.to_excel(os.path.join(path_root, './test_url_error.xlsx'), index=False)
    print('- Error Occurred -')
    print()

df_error if list_series_error else ''

- Error Occurred -



,No,Url,SKU Count,Page,Request_Url
0,1,https://toyota.oempartsonline.com/search?searc...,12123,3,https://toyota.oempartsonline.com/search?searc...


In [11]:
print('Done ~')

Done ~
